## Summary Data for GraphML Files

In [1]:
import graph_tool.all as gt
import os
import numpy as np
import graph_tool
from tqdm import tqdm_notebook

from lib.analysis import clustering
from lib.analysis import dist
from lib.analysis import model
from lib.analysis import power
from lib.analysis import stats

## Part 1: Summary Statistics

Basic statistics for each country's government

In [31]:
data = 'data'
ignore = ['.DS_Store', 'us']
graphs = {}

In [3]:
for country in [d for d in os.listdir(data) if d not in ignore]:
    path = os.path.join(data, country)
    if country not in nets:
        nets[country] = []
    for dir_ in [f for f in os.listdir(path) if f not in ignore]:
        if not dir_.endswith('.graphml'):
            net_path = os.path.join(path, dir_)
            for net in [f for f in os.listdir(net_path) if f not in ignore]:
                if net.endswith('.graphml'):
                    nets[country].append((net[:-8], gt.load_graph(os.path.join(net_path, net))))

In [4]:
for country, graphs in nets.items():
    with open(f'summary.{country}.csv', 'w+') as f:
        f.write(','.join(['Number of Edges', 'Number of Vertices',
                  'Clustering Coefficient', 'Average Degree',
                  'Average Excess Degree', 'Average Distance',
                  'Size of LCC', 'Number of CC']) + '\n')
        for name, graph in graphs:
            f.write(f'{name},{str(stats.summary(graph))}\n')

net_se
net_fr_se
net_fr_an
net_ch_cs
net_ch_cn
uk
net_dk
net_at


## Part 2: Assortivity

**Computes assoritivty on specified attribute for every country across years**

In [2]:
individual_graphs = {}
root = 'data/'
countries = os.listdir(root)
nets = [country + "/" + net + "/" for country in countries
                            for net in os.listdir(root + country)
                            if not net.endswith('.graphml')]
for net in nets:
    for x in os.listdir(root+net):
        if x.endswith('.graphml'):
            individual_graphs[x] = gt.load_graph(root + net + x) 

In [3]:
attribute = 'party'
label = 'party_id'
with open("assortivity_summary.csv", "w+") as f:
    f.write(','.join(['Assortivity Measure']) + '\n')
    for name, g in individual_graphs.items():
        if attribute in g.vertex_properties:
            g.set_directed(False)
            gt.remove_parallel_edges(g)
            stats.generate_party_property(g, attribute)
            f.write(f'{name},{str(stats.assortativity_summary(g, label))}\n')


**Produces assortativity graphs for every country** 
* !!!!!!!!! Please do not run this again, Im autistic and manually moved each pdf into its subdirectory

In [4]:
for name, g in individual_graphs.items():
    name = name.split('.')[0]
    if label in g.vertex_properties:
        gt.graph_draw(g, vertex_fill_color = g.vertex_properties['party_id'], output = "assortativity_" + name + ".pdf" )